In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import keras 
from keras.layers import Dense
import json 
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
import unicodedata
from sklearn.model_selection import train_test_split

In [2]:
file_path = './dialogs.txt'

In [3]:
# file = open(file_path,'r+')
# for i in file:
#     print(i.split('\t'))

In [4]:
d_list = ['hi, how are you doing?', "i'm fine. how about yourself?\n"]
d_list

['hi, how are you doing?', "i'm fine. how about yourself?\n"]

In [5]:
def extraStringRemover(listing):
    strReturn = {"question":'',"ans":""}
    count = 0
    for i in listing:
        #print(count)
        a = re.sub('\s',' ',i)
        if count == 0:
            strReturn['question'] += a.strip()
        else:
            strReturn['ans'] += a.strip()
        #print(a)
        count+=1
    return strReturn

In [6]:
check1 = extraStringRemover(d_list)
print(check1)

{'question': 'hi, how are you doing?', 'ans': "i'm fine. how about yourself?"}


In [7]:
questions = []
answers = []
QnAs = 0

In [8]:
with open(file_path,'r') as f :
    for line in f :
        row = line.split('\t')
        returnStr = extraStringRemover(row)
        print(returnStr)
        questions.append(returnStr['question'])
        answers.append(returnStr['ans'])
        QnAs += 1

{'question': 'hi, how are you doing?', 'ans': "i'm fine. how about yourself?"}
{'question': "i'm fine. how about yourself?", 'ans': "i'm pretty good. thanks for asking."}
{'question': "i'm pretty good. thanks for asking.", 'ans': 'no problem. so how have you been?'}
{'question': 'no problem. so how have you been?', 'ans': "i've been great. what about you?"}
{'question': "i've been great. what about you?", 'ans': "i've been good. i'm in school right now."}
{'question': "i've been good. i'm in school right now.", 'ans': 'what school do you go to?'}
{'question': 'what school do you go to?', 'ans': 'i go to pcc.'}
{'question': 'i go to pcc.', 'ans': 'do you like it there?'}
{'question': 'do you like it there?', 'ans': "it's okay. it's a really big campus."}
{'question': "it's okay. it's a really big campus.", 'ans': 'good luck with school.'}
{'question': 'good luck with school.', 'ans': 'thank you very much.'}
{'question': "how's it going?", 'ans': "i'm doing well. how about you?"}
{'quest

In [9]:
print(f"Total QnA's {QnAs}")

Total QnA's 3725


In [10]:
questions[:10]

['hi, how are you doing?',
 "i'm fine. how about yourself?",
 "i'm pretty good. thanks for asking.",
 'no problem. so how have you been?',
 "i've been great. what about you?",
 "i've been good. i'm in school right now.",
 'what school do you go to?',
 'i go to pcc.',
 'do you like it there?',
 "it's okay. it's a really big campus."]

In [11]:
answers[:10]

["i'm fine. how about yourself?",
 "i'm pretty good. thanks for asking.",
 'no problem. so how have you been?',
 "i've been great. what about you?",
 "i've been good. i'm in school right now.",
 'what school do you go to?',
 'i go to pcc.',
 'do you like it there?',
 "it's okay. it's a really big campus.",
 'good luck with school.']

In [12]:
df = pd.DataFrame({'Question':questions,'Answer':answers})
df.head()

,Question,Answer
0,"hi, how are you doing?",i'm fine. how about yourself?
1,i'm fine. how about yourself?,i'm pretty good. thanks for asking.
2,i'm pretty good. thanks for asking.,no problem. so how have you been?
3,no problem. so how have you been?,i've been great. what about you?
4,i've been great. what about you?,i've been good. i'm in school right now.


In [24]:
contractions = {
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is",
"i'd": "I had / I would",
"i'd've": "I would have",
"i'll": "I shall / I will",
"i'll've": "I shall have / I will have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

In [28]:
contractions["ain't"]

'am not / are not'

In [31]:
str1 = "I ain't you?"
str1

"I ain't you?"

In [37]:
def wordFinder(val):
    for i in val:
        if i in contractions:
            print(i)

In [38]:
wordFinder(str1.lower().split(' '))

ain't
